In [ ]:
# General Imports
import pandas as pd
import matplotlib.pyplot as plt
import random
import geopandas
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
from torch.autograd import Variable

import warnings
import random
import time
from tifffile import imsave

import skimage
from skimage.transform import rescale
import cv2

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

import wandb

In [1]:
def minmax_percentile(img,perc=2):
  lower = np.percentile(img,perc)
  upper = np.percentile(img,100-perc)
  img[img>upper] = upper
  img[img<lower] = lower
  return(img-np.min(img) ) / (np.max(img)-np.min(img))

def plot_tensors(a,b,title="Title"):
  a = a.cpu().detach().numpy()[0]
  b = b.cpu().detach().numpy()[0]

  a = np.transpose(a,(1,2,0))
  b = np.transpose(b,(1,2,0))
  
  a,b = minmax_percentile(a),minmax_percentile(b)
  
  # plot
  fig, axs = plt.subplots(2, 2,figsize=(8,8))
  axs[0,0].imshow(a)
  axs[0,0].set_title("Spot-6")
  axs[0,1].imshow(b)
  axs[0,1].set_title("Sentinel-2")
  
  # Display histogram
  axs[1,0].set_title("Spot-6\n hist. & PDF")
  axs[1,0].hist(np.ravel(a),bins=50, histtype='bar', color='blue')#,weights=100*counts_zi/counts_zi.sum())
  axs[1,0].set_xlabel('Pixel intensity')
  axs[1,0].set_xlim(0, 1)
  #axs[1,0].set_ylim(0, 11000)

  # Display cumulative distribution
  img_cdf, bins = exposure.cumulative_distribution(a, 50)
  axs[1,0].plot(bins, np.multiply(img_cdf,14000), 'r')

  # Display histogram
  axs[1,1].set_title("Sentinel-2\n hist. & PDF")
  axs[1,1].hist(np.ravel(b),bins=50, histtype='bar', color='blue')#,weights=100*counts_zi/counts_zi.sum())
  axs[1,1].set_xlabel('Pixel intensity')
  axs[1,1].set_xlim(0, 1)
  #axs[1,0].set_ylim(0, 100)

  # Display cumulative distribution
  img_cdf, bins = exposure.cumulative_distribution(b, 50)
  axs[1,1].plot(bins, np.multiply(img_cdf,1000), 'r')

  plt.suptitle(title)
  plt.show()
